In [1]:
from lmfit import minimize, Parameters
import kernel as kern
import numpy as np
import pandas as pd
import plotly.graph_objects as go

class AU:
    meter = 5.2917720859e-11 # atomic unit of length in meters
    nm = 5.2917721e-2 # atomic unit of length in nanometres
    second = 2.418884328e-17 # atomic unit of time in seconds
    fs = 2.418884328e-2 # atomic unit of time in femtoseconds
    Joule = 4.359743935e-18 # atomic unit of energy in Joules
    eV = 27.21138383 # atomic unit of energy in electronvolts
    Volts_per_meter = 5.142206313e+11 # atomic unit of electric field in V/m
    Volts_per_Angstrom = 51.42206313 # atomic unit of electric field in V/Angström
    speed_of_light = 137.035999 # vacuum speed of light in atomic units
    Coulomb = 1.60217646e-19 # atomic unit of electric charge in Coulombs
    PW_per_cm2_au = 0.02849451308 # PW/cm^2 in atomic units
AtomicUnits=AU

IpEV_dict = {
    "H": 0.5*27.2114, "He": 24.587, "He_Hacc5": 24.59, "Ne": 21.564, "Kr": 14.0, "Xe": 12.13,
    "Ar": 15.7596, "H2_Hacc5": 15.439, "CO_Hacc5": 14.3, "N2_Hacc5": 14.531, "Rb": 4.1771,
    "Cs": 3.8939, "Fr": 4.0727, "K": 4.3407, 'SiO2': 8.96
}

αPol_dict = {
    "H": 4.51, "He": 1.38, "Ne": 2.66, "Kr": 16.8, "Xe": 27.3, "Ar": 11.1,
    "He_Hacc5": 1.38, "H2_Hacc5": 0, "CO_Hacc5": 13.08, "N2_Hacc5": 11.5, "Rb": 320,
    "Cs": 401, "Fr": 318, "K": 290, "SiO2": 6.16
}

## get input data to fit against

In [53]:
import glob
from scipy.integrate import simpson
from field_functions import LaserField


file_params = [
    {"name":"850nm_350nm_1.25e+14", "Pump_wavel": 850, "Pump_Int": 1.25e14, "Probe_wavel": 350, "Probe_Int": 1e10, "Probe_FWHM": 0.93/AtomicUnits.fs},
    {"name": "850nm_350nm_7.50e+13", "Pump_wavel": 850, "Pump_Int": 7.50e13, "Probe_wavel": 350, "Probe_Int": 6.00e09, "Probe_FWHM": 0.93/AtomicUnits.fs},
    #{"name": "900nm_320nm_5.00e+14", "Pump_wavel": 900, "Pump_Int": 5e14, "Probe_wavel": 320, "Probe_Int": 4e10, "Probe_FWHM": 0.75/AtomicUnits.fs},
    # {"name": "1200nm_320nm_1.00e+14", "Pump_wavel": 1200, "Pump_Int": 1e14, "Probe_wavel": 320, "Probe_Int": 4e10, "Probe_FWHM": 0.75/AtomicUnits.fs},
    #{"name": "1200nm_320nm_5.00e+14", "Pump_wavel": 1200, "Pump_Int": 5e14, "Probe_wavel": 320, "Probe_Int": 4e10, "Probe_FWHM": 0.75/AtomicUnits.fs},
]
tmpFrame=pd.DataFrame()
for currentFile in file_params:
    tmp=pd.read_csv(f"/home/user/TIPTOE/plot_ion_tau_calc_output_data/ion_prob_QS_NA_{currentFile['name']}.csv")
    TipToe_data=pd.DataFrame({"delay":tmp.delay, 
                            "Y":tmp.ion_y})
    TipToe_data["Function_to_fit"]="IonProb"
    pulses=[]
    for dat in TipToe_data.iterrows():
        dat=dat[1]
        # pulse=LaserField()
        # pulse.add_pulse(central_wavelength=currentFile["Pump_wavel"], peak_intensity=currentFile["Pump_Int"], CEP=0, FWHM=currentFile["Pump_wavel"]*1/ AtomicUnits.nm / AtomicUnits.speed_of_light, t0=0)
        # pulse.add_pulse(central_wavelength=currentFile["Probe_wavel"], peak_intensity=currentFile["Probe_Int"], CEP=-np.pi/2, FWHM=currentFile["Probe_FWHM"], t0=-dat.delay)
        laser_pulses = LaserField()
        tau=dat.delay
        laser_pulses.add_pulse(currentFile["Pump_wavel"], currentFile["Pump_Int"], 0, currentFile["Pump_wavel"] / AtomicUnits.nm / AtomicUnits.speed_of_light)
        laser_pulses.add_pulse(currentFile["Probe_wavel"], currentFile["Probe_Int"], -np.pi/2, currentFile["Probe_FWHM"], t0=-tau)
        pulses.append(laser_pulses)
    TipToe_data["pulses"]=pulses
    TipToe_data["Function_to_fit"]="IonProb"
    tmpFrame=pd.concat([tmpFrame, TipToe_data])
TipToe_data=tmpFrame

# Fitting

## define models for Kernel, rate and ionization probability.
### The residual function needs to be defined appropriately for the use-case

##### provide list of indipendent variables that are NOT fit parameters and create corresponding models

In [30]:
%load_ext autoreload
%autoreload
independent_vars = ['t_grid', 'T_grid', 'I', 'wavel', 'cep', 'fwhmau']
# Kernel = Model(kern.Kernel_jit, independent_vars=independent_vars)
# Rate = Model(kern.IonRate, independent_vars=independent_vars)
# @Script: kernels.py(kern.IonProb, independent_vars=independent_vars)

from lmfit import Model

# @Last Modified By: Manoram AgarwalIonRate", "analyticalRate", "IonProb"]
# @Last Modified At: 2024-07-21 16:40:43den_vars=None, params=None):

IonProb=np.vectorize(kern.IonProb)
def model_unified(params, data):
    from field_functions import LaserField
    try:
        params=params.valuesdict()
    except:
        print("")
    Function_to_fit=data.Function_to_fit.to_numpy()
    gamma=params['gamma']
    Multiplier,  e0, a0, a1, b0, b1, b2, p1, d1, Ip, αPol= params['Multiplier'], gamma/2*params['e0'], 4*gamma*params['a0'], params['a1'], 4*gamma**2*params['b0'], 2*gamma*params['b1']*params['a1'], params['b2'], params['p1'], params['d1'], params['Ip'], params['αPol']
    model=np.zeros(Function_to_fit.shape)
    
    if np.any(Function_to_fit) == "Log_QSRate":
        model = kern.QSRate(data.field.to_numpy(), Multiplier,  e0, a0, a1, b0, b1, b2, p1, d1, Ip, αPol)
        #print(model)
        model = np.log(model) # calculate QS rate at peak, c2 is not required here
    elif np.any(Function_to_fit) == "QSRate":
        model = kern.QSRate(data.field.to_numpy(), Multiplier,  e0, a0, a1, b0, b1, b2, p1, d1, Ip, αPol)
    else:
        pulses=data.pulses
        if np.any(Function_to_fit) == "Kernel":
            c2=-16/9*params['b2']*params['c2']
            model = kern.Kernel_jit(data.t_grid, data.T_grid, pulses, Multiplier,  e0, a0, a1, b0, b1, b2, c2, p1, d1, Ip, αPol)
        if np.any(Function_to_fit) == "IonRate":
            c2=-16/9*params['b2']*params['c2']
            model = kern.IonRate(data.t_grid, pulses, Multiplier,  e0, a0, a1, b0, b1, b2, c2, p1, d1, Ip, αPol, dT=0.25)
        if np.any(Function_to_fit) == "IonProb":
            c2=-16/9*params['b2']*params['c2']
            # print(e0, a0, a1, b0, b1, b2, c2, p1, d1, Ip, αPol)
            model = 1-np.exp(-IonProb(pulses, Multiplier,  e0, a0, a1, b0, b1, b2, c2, p1, d1, Ip, αPol))
    #print(model)
    return np.nan_to_num(model)


def residuals(params, data_Nadiabatic=None, uncertainty_adiabatic=None, data_QS=None, uncertainty_QS=None):
    """this function will calculate the residuals automatically providing the right input data for a given function
    prams is a special object that allows manipulation of fit parameters, including fixing them and providing a mathematical expression based on another new parameter

    Args:
        params (lmfit.Parameters()): a dictionary like object containing all information about the fit parameters like bounds, initial guess, fixed mathematical expressions etc. 
        x (an array of dictionaryies of lenght(data)): contains all input data relevant for obtaining data given parameters
        data (a 1d array of numbers with corresponding x): the data can combine numbers from different funcitons as long as the corresponding x is the same
        uncertainty (array, len(data)): to be used to weight the dataset according to needs
    """
    res_QS=[0.]
    res_Nad=[0.]
    if data_Nadiabatic is not None:
        res_Nad= (model_unified(params, data_Nadiabatic)-data_Nadiabatic.Y.to_numpy())/uncertainty_adiabatic
    if data_QS is not None:
        res_QS= (model_unified(params, data_QS)-data_QS.Y.to_numpy())/uncertainty_QS
    return np.concatenate((res_QS,res_Nad))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## fit for Numerical QS rates

In [70]:
gasAr = ["H", "Ne", "He", "N2_Hacc5", "He_Hacc5", "H2_Hacc5", "CO_Hacc5", "Cs", "K", "Fr", "Rb"]
for gas in gasAr:
    params = Parameters()
    # add with tuples: (NAME VALUE  VARY   MIN   MAX  EXPR  BRUTE_STEP)
    params.add('Multiplier', min=1e-3, max=1e3, vary=True, expr=None, brute_step=None)
    IpInit=IpEV_dict.get(gas)/27.2114
    params.add('Ip', value=IpInit, min=IpInit*0.9, vary=False, max=IpInit*1.1, expr=None, brute_step=None)
    params.add('αPol', value= αPol_dict.get(gas), vary=False, expr=None, brute_step=None)
    gammaInit=3/2/params['Ip'].value
    params.add('gamma', value=gammaInit, vary=True, min=gammaInit*0.1, max=gammaInit*2) 
    # gamma is a common parameter, usually used to simultaneously change all vars
    # the following variables are all defined via gamma in the funciton residuals. And the fit parameters here act as more precise tuning tool
    params.add('e0', value=1, vary=True, brute_step=None, min=0., max=10)
    params.add('a0', value=1, min=0.9, vary=False, max=1.1, expr=None, brute_step=None)
    params.add('a1', value=1+0*(params["gamma"]/2-0.25), vary=False, expr=None, brute_step=None)
    params.add('b0', value=1+0*(-1+4*params['Ip']-5*params['Ip']**2+2*params['Ip']**3), vary=False)
    params.add('b1', value=1, min=0.9, vary=False, max=1.1, expr=None, brute_step=None)
    params.add('b2', value=1, min=0.9, max=1.1, vary=False, expr=None, brute_step=None)
    params.add('p1', value=3.5, min=2.5, vary=False, max=4.5, expr=None, brute_step=None)
    params.add('d1',  expr='0.5/gamma')
    ArminRate=pd.read_table(f"/home/manoram/Mathematica/{gas}.rate", comment='#', sep=" +", engine='python', names=["field", "Y"], usecols=[0,1], skiprows=1)
    ArminRate["Function_to_fit"]="Log_QSRate"
    dataQS=ArminRate[ArminRate.field<=4*params['Ip']/4] #only field 4 times the ABI limit
    #dataQS["Function_to_fit"]="Log_QSRate"
    uncertaintyQS=dataQS.Y.to_numpy()*np.linspace(1e-2,0.05, len(dataQS.field))
    ## create array of dictionaries as x
    out = minimize(residuals, params, args=(None, None, dataQS, uncertaintyQS), method='dual_annealing', nan_policy='omit')
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=ArminRate.field, y=ArminRate.Y, mode='lines+markers', name='numerical dataQS', line=dict(color='red')))
    fig.add_trace(go.Scatter(x=ArminRate.field, y=model_unified(out.params, ArminRate), mode='lines', name='fit QS', line=dict(color='blue')))
    fig.update_layout(title=rf'Gas={gas}',
                              xaxis_title='Field Strength',
                              yaxis_title='Log(Rate)')
    fig.show()
    print(out.params.valuesdict())
    out.params.pretty_print()
    #break
    

{'Multiplier': 0.4386088877184498, 'Ip': 0.5, 'αPol': 4.51, 'gamma': 1.8870123015328923, 'e0': 4.111296042707333, 'a0': 1, 'a1': 1.0, 'b0': 1.0, 'b1': 1, 'b2': 1, 'p1': 3.5, 'd1': 0.26496912584715576}
Name           Value      Min      Max   Stderr     Vary     Expr Brute_Step
Ip               0.5     0.45     0.55     None    False     None     None
Multiplier    0.4386    0.001     1000     None     True     None     None
a0                 1      0.9      1.1     None    False     None     None
a1                 1     -inf      inf     None    False     None     None
b0                 1     -inf      inf     None    False     None     None
b1                 1      0.9      1.1     None    False     None     None
b2                 1      0.9      1.1     None    False     None     None
d1             0.265     -inf      inf     None    False 0.5/gamma     None
e0             4.111        0       10     None     True     None     None
gamma          1.887      0.3        6     Non

{'Multiplier': 0.22231835546551748, 'Ip': 0.792461982845425, 'αPol': 2.66, 'gamma': 1.2793787330153525, 'e0': 2.9239078657468824, 'a0': 1, 'a1': 1.0, 'b0': 1.0, 'b1': 1, 'b2': 1, 'p1': 3.5, 'd1': 0.3908146877051457}
Name           Value      Min      Max   Stderr     Vary     Expr Brute_Step
Ip            0.7925   0.7132   0.8717     None    False     None     None
Multiplier    0.2223    0.001     1000     None     True     None     None
a0                 1      0.9      1.1     None    False     None     None
a1                 1     -inf      inf     None    False     None     None
b0                 1     -inf      inf     None    False     None     None
b1                 1      0.9      1.1     None    False     None     None
b2                 1      0.9      1.1     None    False     None     None
d1            0.3908     -inf      inf     None    False 0.5/gamma     None
e0             2.924        0       10     None     True     None     None
gamma          1.279   0.1893  

{'Multiplier': 0.692211325582528, 'Ip': 0.9035551276303314, 'αPol': 1.38, 'gamma': 0.725225876709, 'e0': 4.99606402117029, 'a0': 1, 'a1': 1.0, 'b0': 1.0, 'b1': 1, 'b2': 1, 'p1': 3.5, 'd1': 0.6894403744512652}
Name           Value      Min      Max   Stderr     Vary     Expr Brute_Step
Ip            0.9036   0.8132   0.9939     None    False     None     None
Multiplier    0.6922    0.001     1000     None     True     None     None
a0                 1      0.9      1.1     None    False     None     None
a1                 1     -inf      inf     None    False     None     None
b0                 1     -inf      inf     None    False     None     None
b1                 1      0.9      1.1     None    False     None     None
b2                 1      0.9      1.1     None    False     None     None
d1            0.6894     -inf      inf     None    False 0.5/gamma     None
e0             4.996        0       10     None     True     None     None
gamma         0.7252    0.166     3.32

{'Multiplier': 0.04121520325938956, 'Ip': 0.5340041306217247, 'αPol': 11.5, 'gamma': 5.617934071984035, 'e0': 0.8817268721327964, 'a0': 1, 'a1': 1.0, 'b0': 1.0, 'b1': 1, 'b2': 1, 'p1': 3.5, 'd1': 0.0890006884369541}
Name           Value      Min      Max   Stderr     Vary     Expr Brute_Step
Ip             0.534   0.4806   0.5874     None    False     None     None
Multiplier   0.04122    0.001     1000     None     True     None     None
a0                 1      0.9      1.1     None    False     None     None
a1                 1     -inf      inf     None    False     None     None
b0                 1     -inf      inf     None    False     None     None
b1                 1      0.9      1.1     None    False     None     None
b2                 1      0.9      1.1     None    False     None     None
d1             0.089     -inf      inf     None    False 0.5/gamma     None
e0            0.8817        0       10     None     True     None     None
gamma          5.618   0.2809  

{'Multiplier': 3.6597333578882734, 'Ip': 0.9036653755411335, 'αPol': 1.38, 'gamma': 0.4243905841058913, 'e0': 8.746694626936057, 'a0': 1, 'a1': 1.0, 'b0': 1.0, 'b1': 1, 'b2': 1, 'p1': 3.5, 'd1': 1.1781599750932341}
Name           Value      Min      Max   Stderr     Vary     Expr Brute_Step
Ip            0.9037   0.8133    0.994     None    False     None     None
Multiplier      3.66    0.001     1000     None     True     None     None
a0                 1      0.9      1.1     None    False     None     None
a1                 1     -inf      inf     None    False     None     None
b0                 1     -inf      inf     None    False     None     None
b1                 1      0.9      1.1     None    False     None     None
b2                 1      0.9      1.1     None    False     None     None
d1             1.178     -inf      inf     None    False 0.5/gamma     None
e0             8.747        0       10     None     True     None     None
gamma         0.4244    0.166   

...

{'Multiplier': 0.22163618709285832, 'Ip': 0.5255150414899638, 'αPol': 13.08, 'gamma': 2.433064448398187, 'e0': 3.6785676524221063, 'a0': 1, 'a1': 1.0, 'b0': 1.0, 'b1': 1, 'b2': 1, 'p1': 3.5, 'd1': 0.2055021601787721}
Name           Value      Min      Max   Stderr     Vary     Expr Brute_Step
Ip            0.5255    0.473   0.5781     None    False     None     None
Multiplier    0.2216    0.001     1000     None     True     None     None
a0                 1      0.9      1.1     None    False     None     None
a1                 1     -inf      inf     None    False     None     None
b0                 1     -inf      inf     None    False     None     None
b1                 1      0.9      1.1     None    False     None     None
b2                 1      0.9      1.1     None    False     None     None
d1            0.2055     -inf      inf     None    False 0.5/gamma     None
e0             3.679        0       10     None     True     None     None
gamma          2.433   0.2854 

{'Multiplier': 0.5003935100264955, 'Ip': 0.14309811329075314, 'αPol': 401, 'gamma': 20.964636996327588, 'e0': 2.260251651628675, 'a0': 1, 'a1': 1.0, 'b0': 1.0, 'b1': 1, 'b2': 1, 'p1': 3.5, 'd1': 0.023849685548458857}
Name           Value      Min      Max   Stderr     Vary     Expr Brute_Step
Ip            0.1431   0.1288   0.1574     None    False     None     None
Multiplier    0.5004    0.001     1000     None     True     None     None
a0                 1      0.9      1.1     None    False     None     None
a1                 1     -inf      inf     None    False     None     None
b0                 1     -inf      inf     None    False     None     None
b1                 1      0.9      1.1     None    False     None     None
b2                 1      0.9      1.1     None    False     None     None
d1           0.02385     -inf      inf     None    False 0.5/gamma     None
e0              2.26        0       10     None     True     None     None
gamma          20.96    1.048 

{'Multiplier': 0.41031319598945015, 'Ip': 0.15951770213954444, 'αPol': 290, 'gamma': 18.806690165180733, 'e0': 2.0885269425313933, 'a0': 1, 'a1': 1.0, 'b0': 1.0, 'b1': 1, 'b2': 1, 'p1': 3.5, 'd1': 0.026586283689924075}
Name           Value      Min      Max   Stderr     Vary     Expr Brute_Step
Ip            0.1595   0.1436   0.1755     None    False     None     None
Multiplier    0.4103    0.001     1000     None     True     None     None
a0                 1      0.9      1.1     None    False     None     None
a1                 1     -inf      inf     None    False     None     None
b0                 1     -inf      inf     None    False     None     None
b1                 1      0.9      1.1     None    False     None     None
b2                 1      0.9      1.1     None    False     None     None
d1           0.02659     -inf      inf     None    False 0.5/gamma     None
e0             2.089        0       10     None     True     None     None
gamma          18.81   0.940

{'Multiplier': 0.43652876279009273, 'Ip': 0.14966888877455772, 'αPol': 318, 'gamma': 20.044245832003337, 'e0': 2.187644470705131, 'a0': 1, 'a1': 1.0, 'b0': 1.0, 'b1': 1, 'b2': 1, 'p1': 3.5, 'd1': 0.024944814795759624}
Name           Value      Min      Max   Stderr     Vary     Expr Brute_Step
Ip            0.1497   0.1347   0.1646     None    False     None     None
Multiplier    0.4365    0.001     1000     None     True     None     None
a0                 1      0.9      1.1     None    False     None     None
a1                 1     -inf      inf     None    False     None     None
b0                 1     -inf      inf     None    False     None     None
b1                 1      0.9      1.1     None    False     None     None
b2                 1      0.9      1.1     None    False     None     None
d1           0.02494     -inf      inf     None    False 0.5/gamma     None
e0             2.188        0       10     None     True     None     None
gamma          20.04    1.002

{'Multiplier': 0.44503458237990395, 'Ip': 0.15350551607047047, 'αPol': 320, 'gamma': 19.543271647793922, 'e0': 2.110242042872672, 'a0': 1, 'a1': 1.0, 'b0': 1.0, 'b1': 1, 'b2': 1, 'p1': 3.5, 'd1': 0.025584252678411745}
Name           Value      Min      Max   Stderr     Vary     Expr Brute_Step
Ip            0.1535   0.1382   0.1689     None    False     None     None
Multiplier     0.445    0.001     1000     None     True     None     None
a0                 1      0.9      1.1     None    False     None     None
a1                 1     -inf      inf     None    False     None     None
b0                 1     -inf      inf     None    False     None     None
b1                 1      0.9      1.1     None    False     None     None
b2                 1      0.9      1.1     None    False     None     None
d1           0.02558     -inf      inf     None    False 0.5/gamma     None
e0              2.11        0       10     None     True     None     None
gamma          19.54   0.9772

## fit for ionization probabilities

In [55]:
%load_ext autoreload
%autoreload
from plotly.subplots import make_subplots
gas="H"

# del params
params = Parameters()
tmp={'Multiplier': 0.4, 'Ip': 0.5, 'αPol': 4.51, 'gamma': 3, 'e0': 1, 'a0': 1, 'a1': 1.0, 'b0': 1.0, 'b1': 1, 'b2': 1, 'p1': 3.5, 'd1': 0.22496851712539684, 'c2': 1}
# @Script: field_functions.pyE  VARY   MIN   MAX  EXPR  BRUTE_STEP)
params.add('Multiplier', min=tmp['Multiplier']*0.8, max=tmp['Multiplier']*1.2, value=tmp['Multiplier'], vary=True, expr=None, brute_step=None)
IpInit=IpEV_dict.get(gas)/27.2114
params.add('Ip', value=IpInit, min=IpInit*0.9, vary=False, max=IpInit*1.1, expr=None, brute_step=None)
params.add('αPol', value= αPol_dict.get(gas), vary=False, expr=None, brute_step=None)
# @Last Modified By: Manoram Agarwalget(gas), vary=True, expr=None, brute_step=None)
# @Last Modified At: 2024-08-07 11:13:46
params.add('gamma', value=tmp['gamma'], vary=True, min=tmp['gamma']*0.5, max=tmp['gamma']*2) 
# gamma is a common parameter, usually used to simultaneously change all vars
# the following variables are all defined via gamma in the funciton residuals. And the fit parameters here act as more precise tuning tool
params.add('e0', value=tmp["e0"], vary=True, expr=None, brute_step=None, min=0.5, max=5)
params.add('a0', value=tmp["a0"], vary=False, expr=None, brute_step=None, min=tmp['a0']*0.5, max=tmp['a0']*1.5)
params.add('a1', value=tmp["a1"], vary=False, expr=None, brute_step=None)#, min=0., max=2)
params.add('b0', value=tmp['b0'], vary=False)
params.add('b1', value=tmp["b1"], min=0, vary=False)#tmp['b1']*0.7, vary=True, max=tmp['b1']*1.3, expr=None, brute_step=None)
params.add('b2', value=tmp["b2"], vary=False, expr=None, brute_step=None,min=0)#, min=tmp["b2"]*0.7, max=tmp["b2"]*1.3)
params.add('c2', value=tmp['c2'], vary=True, expr=None, brute_step=None, min=0, max=10)
params.add('p1', value=tmp["p1"], vary=False)#, expr=None, brute_step=None, min=3.2, max=3.8)
params.add('d1', value=tmp["d1"], vary=False, min=0.)


# params.add('Multiplier', value=0.4386088877184498, min=0.10, max=0.9, vary=True, expr=None, brute_step=None)
# IpInit=IpEV_dict.get(gas)/27.2114
# params.add('Ip', value=IpInit, min=IpInit*0.9, vary=False, max=IpInit*1.1, expr=None, brute_step=None)
# params.add('αPol', value= αPol_dict.get(gas), vary=False, expr=None, brute_step=None)
# gammaInit=2 #3/2/params['Ip'].value
# params.add('gamma', value=gammaInit, vary=True, min=gammaInit*0.1, max=gammaInit*3) 
# # gamma is a common parameter, usually used to simultaneously change all vars
# # the following variables are all defined via gamma in the funciton residuals. And the fit parameters here act as more precise tuning tool
# params.add('e0', value=4, vary=True, brute_step=None, min=0., max=10)
# params.add('a0', value=1, min=0.9, vary=False, max=1.1, expr=None, brute_step=None)
# params.add('a1', value=1+0*(params["gamma"]/2-0.25), vary=False, expr=None, brute_step=None)
# params.add('b0', value=1+0*(-1+4*params['Ip']-5*params['Ip']**2+2*params['Ip']**3), vary=False)
# params.add('b1', value=1, min=0.9, vary=False, max=1.1, expr=None, brute_step=None)
# params.add('b2', value=1, min=0.9, max=1.1, vary=False, expr=None, brute_step=None)
# params.add('p1', value=3.5, min=2.5, vary=False, max=4.5, expr=None, brute_step=None)
# params.add('d1',  expr='0.5/gamma')
# params.add('c2', value=1, vary=True, expr=None, brute_step=None, min=0, max=5)



print(params.valuesdict())
print(params.pretty_print())
dataNA=TipToe_data
uncertaintyNA=0.05*dataNA.Y.to_numpy()

def iterator(params, iter, resid, dummy1, dummy2, dummy3, dummy4):
    if iter%5==0 or iter==0:
        print(f"current iteration={iter}", f"residual={np.sum((resid*0.01)**2)}", f"params={params.valuesdict()}", sep='\n')
        fig = go.Figure()
        data=TipToe_data
        fig.add_trace(go.Scatter(x=data.delay, y=data.Y, mode='markers', name='numerical data', line=dict(color='red')))
        fig.add_trace(go.Scatter(x=data.delay, y=model_unified(params, data), mode='markers', name='fit', line=dict(color='blue')))
        fig.update_layout(title=rf'Gas={gas}',
                                    xaxis_title='wavelengths',
                                    yaxis_title='ionization probability')
        fig.show()
        del data
        
    
out = minimize(residuals, params, args=(dataNA, uncertaintyNA, None, None), nan_policy='omit', iter_cb=iterator, method='dual_annealing')
fig = go.Figure()
data=TipToe_data
fig.add_trace(go.Scatter(x=data.delay, y=data.Y, mode='markers', name='numerical data', line=dict(color='red')))
fig.add_trace(go.Scatter(x=data.delay, y=model_unified(params, data), mode='markers', name='fit', line=dict(color='blue')))
fig.update_layout(title=rf'Gas={gas}',
                            xaxis_title='delay',
                            yaxis_title='ionization probability')
fig.show()
out.params.pretty_print()
print(out.params.valuesdict())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'Multiplier': 0.4, 'Ip': 0.5, 'αPol': 4.51, 'gamma': 3, 'e0': 1, 'a0': 1, 'a1': 1.0, 'b0': 1.0, 'b1': 1, 'b2': 1, 'c2': 1, 'p1': 3.5, 'd1': 0.22496851712539684}
Name           Value      Min      Max   Stderr     Vary     Expr Brute_Step
Ip               0.5     0.45     0.55     None    False     None     None
Multiplier       0.4     0.32     0.48     None     True     None     None
a0                 1      0.5      1.5     None    False     None     None
a1                 1     -inf      inf     None    False     None     None
b0                 1     -inf      inf     None    False     None     None
b1                 1        0      inf     None    False     None     None
b2                 1        0      inf     None    False     None     None
c2                 1        0       10     None     True     None     None
d1             0.225        0      inf     None    False     None     Non

current iteration=10
residual=0.020438937566758443
params={'Multiplier': 0.41211652632465756, 'Ip': 0.5, 'αPol': 4.51, 'gamma': 2.586940199136734, 'e0': 2.938316985964775, 'a0': 1, 'a1': 1.0, 'b0': 1.0, 'b1': 1, 'b2': 1, 'c2': 0.5315412282943726, 'p1': 3.5, 'd1': 0.22496851712539684}


current iteration=15
residual=8.29729339189516
params={'Multiplier': 0.32000000000000006, 'Ip': 0.5, 'αPol': 4.51, 'gamma': 1.5, 'e0': 0.5, 'a0': 1, 'a1': 1.0, 'b0': 1.0, 'b1': 1, 'b2': 1, 'c2': 0.0, 'p1': 3.5, 'd1': 0.22496851712539684}


In [27]:
params.pretty_print()

Name           Value      Min      Max   Stderr     Vary     Expr Brute_Step
Ip               0.5     0.45     0.55     None    False     None     None
Multiplier    0.3538    0.283   0.4245     None    False     None     None
a0                 1      0.5      1.5     None    False     None     None
a1                 1     -inf      inf     None    False     None     None
b0                 1     -inf      inf     None    False     None     None
b1                 1        0      inf     None     True     None     None
b2                 1        0      inf     None    False     None     None
c2               2.5       -5       10     None     True     None     None
d1             0.225        0      inf     None    False     None     None
e0             3.754        0      inf     None    False     None     None
gamma          2.223        2    2.445     None    False     None     None
p1               3.5     -inf      inf     None    False     None     None
αPol            4.51   

In [ ]:
print(out.params.valuesdict())

{'Multiplier': 0.6165745119213167, 'Ip': 0.5, 'αPol': 4.51, 'gamma': 1.656383091791583, 'e0': 5.000000000000009, 'a0': 1, 'a1': 1.0, 'b0': 1.0, 'b1': 1, 'b2': 1, 'p1': 3.5, 'd1': 0.30186253559204607, 'c2': 4.9999999999999885}


In [79]:
print(out.params.valuesdict())
{'Multiplier': 0.13344736091053536, 'Ip': 0.5, 'αPol': 4.51, 'gamma': 4.0, 'e0': 1.5671052947385071, 'a0': 1, 'a1': 1.0, 'b0': 1.0, 'b1': 1, 'b2': 1, 'p1': 3.5, 'd1': 0.125, 'c2': 2.0}

{'Multiplier': 0.3537672860419956, 'Ip': 0.5, 'αPol': 4.51, 'gamma': 2.2225332077079094, 'e0': 3.7538886827392335, 'a0': 1, 'a1': 1.0, 'b0': 1.0, 'b1': 1, 'b2': 1, 'p1': 3.5, 'd1': 0.22496851712539684, 'c2': 5.0}


{'Multiplier': 0.13344736091053536,
 'Ip': 0.5,
 'αPol': 4.51,
 'gamma': 4.0,
 'e0': 1.5671052947385071,
 'a0': 1,
 'a1': 1.0,
 'b0': 1.0,
 'b1': 1,
 'b2': 1,
 'p1': 3.5,
 'd1': 0.125,
 'c2': 2.0}

In [37]:
%load_ext autoreload
%autoreload
#params={'Multiplier': 0.281726606169747, 'Ip': 0.5, 'αPol': 4.51, 'gamma': 3.0, 'e0': 2.0, 'a0': 1, 'a1': 1.25, 'b0': 0.0, 'b1': 1, 'b2': 5.279312202389168, 'c2': 0.14300061958645038, 'p1': 3.5, 'd1': 0.22769889212516264}
params=out.params.valuesdict()
gas="H"
ArminRate=pd.read_table(f"/home/manoram/Mathematica/{gas}.rate", comment='#', sep=" +", engine='python', names=["field", "Y"], usecols=[0,1], skiprows=1)
ArminRate["Function_to_fit"]="Log_QSRate"
fig = go.Figure()
fig.add_trace(go.Scatter(x=ArminRate.field, y=ArminRate.Y, mode='lines+markers', name='numerical data', line=dict(color='red')))
fig.add_trace(go.Scatter(x=ArminRate.field, y=model_unified(out.params, ArminRate), mode='lines', name='numerical data', line=dict(color='blue')))
fig.update_layout(title=rf'Gas={gas}',
                        xaxis_title='Field Strength',
                        yaxis_title='Log(Rate)')
fig.show()
data=tRecX_data
data["Function_to_fit"]="IonProb"
groups=data.groupby(["intens", "cep", "FWHM_OC"])
for name1, group1 in groups:
    # print(name1)
    # print(group1.wavel)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=group1.wavel, y=group1.Y, mode='lines+markers', name='numerical data', line=dict(color='red')))
    fig.add_trace(go.Scatter(x=group1.wavel, y=model_unified(params, group1), mode='lines+markers', name='fit', line=dict(color='blue')))
    fig.update_layout(title=rf'Intensity={name1[0]:.2e} W/cm2, cep={name1[1]/np.pi*180:.0f} deg, FWHM_OC={name1[2]:.0f}',
                                xaxis_title='wavelengths',
                                yaxis_title='ionization probability')
    fig.show()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


FileNotFoundError: [Errno 2] No such file or directory: '/home/manoram/Mathematica/H.rate'

## Fit for solids

In [69]:
E0 = np.array([0.00972345, 0.0194469,  0.02917036, 0.03889381]) # peak electric field in atomic units
omega0 = np.array([0.0650905,  0.07593892, 0.0911267,  0.11390838]) # atomic units
fwhm = 2*np.pi / omega0
n_e = np.array([[8.99784392e-11, 2.53996521e-10, 1.07581469e-09, 6.59793050e-09],
 [4.20348094e-08, 6.67472929e-08, 1.36042333e-07, 3.23495614e-07],
 [9.81743929e-07, 1.21367327e-06, 1.68519371e-06, 2.80512395e-06],
 [7.35410652e-06, 8.18809011e-06, 1.03745443e-05, 1.40791601e-05]]) # n_e(E0, omega0)
df=pd.DataFrame(n_e, columns=omega0, index=E0)
df=df.stack().reset_index(name='Y').rename(columns={'level_0':'E0', 'level_1': 'omega0'})
print("for consistancy please check that the 2d array was translated correctly into the desired data format")
df["wavel"]=137.035999/(df.omega0/2/np.pi)*5.2917721e-2 
df["intens"]= df.E0**2*1e15/0.02849451308
df["fwhmau"]=2*np.pi/df.omega0
df["FWHM_OC"]= df.fwhmau*df.omega0/2/np.pi
df["cep"]= 0
SiO2_data=df
print(SiO2_data)

for consistancy please check that the 2d array was translated correctly into the desired data format
          E0    omega0             Y       wavel        intens     fwhmau  \
0   0.009723  0.065090  8.997844e-11  700.000037  3.318024e+12  96.529990   
1   0.009723  0.075939  2.539965e-10  600.000006  3.318024e+12  82.739988   
2   0.009723  0.091127  1.075815e-09  500.000027  3.318024e+12  68.949993   
3   0.009723  0.113908  6.597930e-09  400.000004  3.318024e+12  55.159992   
4   0.019447  0.065090  4.203481e-08  700.000037  1.327210e+13  96.529990   
5   0.019447  0.075939  6.674729e-08  600.000006  1.327210e+13  82.739988   
6   0.019447  0.091127  1.360423e-07  500.000027  1.327210e+13  68.949993   
7   0.019447  0.113908  3.234956e-07  400.000004  1.327210e+13  55.159992   
8   0.029170  0.065090  9.817439e-07  700.000037  2.986224e+13  96.529990   
9   0.029170  0.075939  1.213673e-06  600.000006  2.986224e+13  82.739988   
10  0.029170  0.091127  1.685194e-06  500.000027  2.

In [82]:
data=SiO2_data
data["Function_to_fit"]="IonProb"
gas="SiO2"
# @Script: kernels.py
# add with tuples: (NAME VALUE  VARY   MIN   MAX  EXPR  BRUTE_STEP)
params.add('Multiplier',  min=1e-6, value=1)#, min=25, max=27, value=26.44, vary=True, expr=None, brute_step=None)
IpInit=IpEV_dict.get(gas)/27.2114
# @Last Modified By: Manoram AgarwalpInit*0.9, vary=False, max=IpInit*1.1, expr=None, brute_step=None)
# @Last Modified At: 2024-07-23 13:24:16gas), vary=False, expr=None, brute_step=None)
gammaInit=3/2/params['Ip'].value
params.add('gamma', value=gammaInit, vary=True, min=gammaInit*0.8, max=gammaInit*1.2) 
# gamma is a common parameter, usually used to simultaneously change all vars
# the following variables are all defined via gamma in the funciton residuals. And the fit parameters here act as more precise tuning tool
params.add('e0', value=1+params["gamma"]/3, vary=True, expr=None, brute_step=None, min=0)
params.add('a0', value=1, vary=True, expr=None, brute_step=None, min=0.8, max=1.2)
params.add('a1', value=params["gamma"]/2-0.25, vary=False, expr=None, brute_step=None)#, min=0., max=1.2)
params.add('b0', value=-1+4*params['Ip']-5*params['Ip']**2+2*params['Ip']**3, vary=True)
params.add('b1', value=1, min=0.9, vary=True, max=1.1, expr=None, brute_step=None)
params.add('b2', min=0.)#, value=6.12, vary=True, expr=None, brute_step=None, min=5., max=8)
params.add('c2', value=1, vary=True, expr=None, brute_step=None, min=0.5, max=2)
params.add('p1', value=3.5, vary=True, expr=None, brute_step=None, min=2.7, max=4.3)
params.add('d1', value=1, min=0.)

x=np.arange(len(data))
uncertainty=0.0001#data.Y*0.01
## create array of dictionaries as x
def iterator(params, iter, resid, dummy1, dummy2, dummy3):
    print(f"current iteration={iter}", f"residual={np.sum((resid*0.01)**2)}", f"params={params.valuesdict()}", sep='\n')
    
out = minimize(residuals, params, args=(x, data, uncertainty), nan_policy='omit', iter_cb=iterator, method='nelder')
out.params.pretty_print()
groups=data.groupby([ "cep", "FWHM_OC"])
for name1, group1 in groups:
    # print(name1)
    # print(group1.wavel)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=group1.wavel, y=group1.Y, mode='lines+markers', name='numerical data', line=dict(color='red')))
    fig.add_trace(go.Scatter(x=group1.wavel, y=model_unified(params, group1), mode='lines+markers', name='fit', line=dict(color='blue')))
    fig.update_layout(title=rf'cep={name1[1]/np.pi*180:.0f} deg, FWHM_OC={name1[2]:.0f}',
                                xaxis_title='wavelengths',
                                yaxis_title='ionization probability')
    fig.show()
#Intensity={name1[0]:.2e} W/cm2, 

current iteration=1
residual=2864.903890051612
params={'Multiplier': 0.9999999999999999, 'Ip': 0.32927376026224303, 'αPol': 6.16, 'gamma': 4.555479910714285, 'e0': 2.5184933035714288, 'a0': 1.0, 'a1': 2.0277399553571427, 'b0': -0.15361048644810626, 'b1': 1.0, 'b2': 0.0, 'c2': 1.0, 'p1': 3.5, 'd1': 0.9999999999999998}
current iteration=2
residual=3245.8182507987417
params={'Multiplier': 1.0754516956324216, 'Ip': 0.32927376026224303, 'αPol': 6.16, 'gamma': 4.555479910714285, 'e0': 2.5184933035714288, 'a0': 1.0, 'a1': 2.0277399553571427, 'b0': -0.15361048644810626, 'b1': 1.0, 'b2': 0.0, 'c2': 1.0, 'p1': 3.5, 'd1': 0.9999999999999998}
current iteration=3
residual=2865.723669372871
params={'Multiplier': 0.9999999999999999, 'Ip': 0.32927376026224303, 'αPol': 6.16, 'gamma': 4.555707684707449, 'e0': 2.5184933035714288, 'a0': 1.0, 'a1': 2.0277399553571427, 'b0': -0.15361048644810626, 'b1': 1.0, 'b2': 0.0, 'c2': 1.0, 'p1': 3.5, 'd1': 0.9999999999999998}
current iteration=4
residual=3188.56912882

IndexError: tuple index out of range